### IPN Troubleshooting Notebook

#### Merchant's IPN Status
IPN delivery failures to merchant endpoints are updated in table "wipn_merchant_status" QNOTIF DB.
The status-key here is as follows:
* O: Normal
* B: Blocked
* D: Disabled

The status here depends on failure_count. <br> If everything is fine and the IPNs are working as expected for the merchant, the status will be 'O'. <br>
When the failure count reaches a threshold of 100, the status automatically updates from Normal ('O') to Blocked ('B'). At this point, no new IPNs will be sent to the merchant, but only retries will happen. <br>
When the failure_count reaches under 95, the status will automatically be set to 'O' from 'B'.<br>
If the status stays as 'B' for 5 consecutive days, it will change to Disabled ('D'), which means that IPNs will be no longer sent to merchant's endpoint but would directly be archived to whttp_msg_queue_archive table. <br>
*Note:* Enabling the IPN in Admin will update the Status here.

In [15]:
%%oracle -db QNOTIF
select * from QTOCDBA.WIPN_MERCHANT_STATUS where merchant = '1897159474893678550'

HTML(value='')

HTML(value='')

ID,MERCHANT,TYPE,STATUS,FAILURE_COUNT,PYPL_TIME_TOUCHED,TIME_DISABLED,DISABLED_BY,WARNING_COUNT
11002733,1897159474893678550,A,D,100,1599629576,1599629576,B,13


HTML(value='')

#### The current IPN queue for the merchant is present in the WHTTP_MSG_QUEUE table
Just using the merchant ID will not be enough to query this table, as it may result in a large number of rows. <br>
To narrow down the results, use the decrypted message ID, or Transaction ID, or Correlation ID. <br>

##### Search by Merchant Account Number and Decrypted Message ID (that we can get from Admin)

In [24]:
%%oracle -db QNOTIF
select * from QTOCDBA.WHTTP_MSG_QUEUE where 
account_number = '1897159474893678550'
and ID = '25348437973' -- decrypted message ID
--and PAYMENT_ID_ENCRYPTED = '7JS09569325339604'  -- this is the transaction_id; uncomment this line if you want to search by transaction id
--and CORRELATION_ID = 'c31e3ca416da9' --cal_id; uncomment this line if you want to search by cal-id

HTML(value='')

HTML(value='')

PARTITION_ID,ID,TYPE,FLAGS,STATUS,TARGET,PRIORITY,TIME_CREATED,TIME_SEND,TIME_LAST,RETRY_INTERVAL,RETRY_COUNT,RETRY_MAX,HTTP_TYPE,HTTP_FLAGS,HTTP_URL,HTTP_PROXY,HTTP_HEADERS,HTTP_BODY,HTTP_RESULT,HTTP_REPLY,ACCOUNT_NUMBER,SLOT,BLOCKED,HTTP_RESPONSE_TIME,HTTP_RESPONSE_CODE,PAYMENT_ID_ENCRYPTED,CORRELATION_ID
6,25348437973,0,0,F,20351004078186965,100,1599280715,1599705075,1599618695,86400,16,16,None,2,https://billing.wzukltd.com/ipn/paypal.php,None,Content-Type: application/x-www-form-urlencodedUser-Agent: PayPal IPN ( https://www.paypal.com/ipn ),b'mc_gross=28.00&mp_custom=&mp_currency=USD&protection_eligibility=Eligible&payer_id=86955BFBTUGJU&payment_date=21%3A38%3A17+Sep+04%2C+2020+PDT&mp_id=B-1GP27012SG5635716&payment_status=Completed&charset=windows-1252&first_name=Latrachaui&mp_status=0&mc_fee=0.78&notify_version=3.9&custom=&payer_status=verified&business=websitebuilder-paypal%40endurance.com&quantity=1&verify_sign=A.GF2oArIMOgrTQolyjukfNVpReOAX5Y3upftZr-xOVJ-Y.RXzyxQ6KE&payer_email=wickedhaircollection20%40gmail.com&txn_id=20F39644D39514533&payment_type=instant&payer_business_name=Wicked+Hair+Collection&last_name=Tyler&mp_desc=&receiver_email=websitebuilder-paypal%40endurance.com&payment_fee=0.78&mp_cycle_start=26&shipping_discount=0.00&insurance_amount=0.00&receiver_id=YSUP7PBK2P5PC&txn_type=merch_pmt&item_name=&discount=0.00&mc_currency=USD&item_number=&residence_country=US&shipping_method=Default&transaction_subject=&payment_gross=28.00&ipn_track_id=c31e3ca416da9',2,None,1897159474893678550,727,B,20002,None,20F39644D39514533,c31e3ca416da9


HTML(value='')

##### Search by Merchant Account Number and the Transaction ID

In [27]:
%%oracle -db QNOTIF
select * from QTOCDBA.WHTTP_MSG_QUEUE where 
account_number = '1897159474893678550'
and PAYMENT_ID_ENCRYPTED = '7JS09569325339604'

HTML(value='')

HTML(value='')

PARTITION_ID,ID,TYPE,FLAGS,STATUS,TARGET,PRIORITY,TIME_CREATED,TIME_SEND,TIME_LAST,RETRY_INTERVAL,RETRY_COUNT,RETRY_MAX,HTTP_TYPE,HTTP_FLAGS,HTTP_URL,HTTP_PROXY,HTTP_HEADERS,HTTP_BODY,HTTP_RESULT,HTTP_REPLY,ACCOUNT_NUMBER,SLOT,BLOCKED,HTTP_RESPONSE_TIME,HTTP_RESPONSE_CODE,PAYMENT_ID_ENCRYPTED,CORRELATION_ID
6,25348428547,0,0,F,20351018578510715,100,1599280595,1599706883,1599620503,86400,16,16,None,2,https://billing.wzukltd.com/ipn/paypal.php,None,Content-Type: application/x-www-form-urlencodedUser-Agent: PayPal IPN ( https://www.paypal.com/ipn ),b'mc_gross=13.80&mp_custom=&mp_currency=USD&protection_eligibility=Eligible&payer_id=BK92QG5V9MFRG&payment_date=21%3A36%3A15+Sep+04%2C+2020+PDT&mp_id=B-0WG205118A308015C&payment_status=Completed&charset=windows-1252&first_name=Stephen&mp_status=0&mc_fee=0.65&notify_version=3.9&custom=&payer_status=verified&business=websitebuilder-paypal%40endurance.com&quantity=1&verify_sign=AJtbBrwSjN45OT2XSb56saV4bE8aAZ3BjxQvC8OertKhq39c-hqM0eZv&payer_email=dispensingoptician%40hotmail.com&txn_id=7JS09569325339604&payment_type=instant&last_name=Bill&mp_desc=&receiver_email=websitebuilder-paypal%40endurance.com&payment_fee=0.65&mp_cycle_start=27&shipping_discount=0.00&insurance_amount=0.00&receiver_id=YSUP7PBK2P5PC&txn_type=merch_pmt&item_name=&discount=0.00&mc_currency=USD&item_number=&residence_country=NZ&shipping_method=Default&transaction_subject=&payment_gross=13.80&ipn_track_id=3380779a829b3',2,None,1897159474893678550,149,B,20001,None,7JS09569325339604,3380779a829b3


HTML(value='')

##### Search by Merchant Account Number and the CAL Correlation ID

In [28]:
%%oracle -db QNOTIF
select * from QTOCDBA.WHTTP_MSG_QUEUE where 
account_number = '1897159474893678550'
and CORRELATION_ID = 'c31e3ca416da9'

HTML(value='')

HTML(value='')

PARTITION_ID,ID,TYPE,FLAGS,STATUS,TARGET,PRIORITY,TIME_CREATED,TIME_SEND,TIME_LAST,RETRY_INTERVAL,RETRY_COUNT,RETRY_MAX,HTTP_TYPE,HTTP_FLAGS,HTTP_URL,HTTP_PROXY,HTTP_HEADERS,HTTP_BODY,HTTP_RESULT,HTTP_REPLY,ACCOUNT_NUMBER,SLOT,BLOCKED,HTTP_RESPONSE_TIME,HTTP_RESPONSE_CODE,PAYMENT_ID_ENCRYPTED,CORRELATION_ID
6,25348437973,0,0,F,20351004078186965,100,1599280715,1599705075,1599618695,86400,16,16,None,2,https://billing.wzukltd.com/ipn/paypal.php,None,Content-Type: application/x-www-form-urlencodedUser-Agent: PayPal IPN ( https://www.paypal.com/ipn ),b'mc_gross=28.00&mp_custom=&mp_currency=USD&protection_eligibility=Eligible&payer_id=86955BFBTUGJU&payment_date=21%3A38%3A17+Sep+04%2C+2020+PDT&mp_id=B-1GP27012SG5635716&payment_status=Completed&charset=windows-1252&first_name=Latrachaui&mp_status=0&mc_fee=0.78&notify_version=3.9&custom=&payer_status=verified&business=websitebuilder-paypal%40endurance.com&quantity=1&verify_sign=A.GF2oArIMOgrTQolyjukfNVpReOAX5Y3upftZr-xOVJ-Y.RXzyxQ6KE&payer_email=wickedhaircollection20%40gmail.com&txn_id=20F39644D39514533&payment_type=instant&payer_business_name=Wicked+Hair+Collection&last_name=Tyler&mp_desc=&receiver_email=websitebuilder-paypal%40endurance.com&payment_fee=0.78&mp_cycle_start=26&shipping_discount=0.00&insurance_amount=0.00&receiver_id=YSUP7PBK2P5PC&txn_type=merch_pmt&item_name=&discount=0.00&mc_currency=USD&item_number=&residence_country=US&shipping_method=Default&transaction_subject=&payment_gross=28.00&ipn_track_id=c31e3ca416da9',2,None,1897159474893678550,727,B,20002,None,20F39644D39514533,c31e3ca416da9


HTML(value='')

##### A blocked 'B' status in the above result would mean that the IPN notification was not sent, and consequently, you won't see a HTTP_RESPONSE_CODE against it.

#### The IPN Archived Queue is present in the WHTTP_MSG_QUEUE_ARCHIVE table

Let us query the IPN archive table. Both successful, and unsent (blocked; if the merchant's IPN is in 'Disabled' status) IPNs will land here.

##### Search by Merchant Account Number and the Decrypted Message ID (that we can get from Admin)

In [25]:
%%oracle -db QNOTIF
select * from QTOCDBA.WHTTP_MSG_QUEUE_ARCHIVE where account_number = '1897159474893678550' and id = '25348437973'

HTML(value='')

HTML(value='')

HTML(value='')

PARTITION_ID,ID,TYPE,FLAGS,STATUS,PRIORITY,TIME_CREATED,TIME_SEND,TIME_LAST,RETRY_INTERVAL,RETRY_COUNT,RETRY_MAX,HTTP_TYPE,HTTP_FLAGS,HTTP_URL,HTTP_PROXY,HTTP_HEADERS,HTTP_BODY,HTTP_RESULT,HTTP_REPLY,ACCOUNT_NUMBER,HTTP_RESPONSE_TIME,HTTP_RESPONSE_CODE,PAYMENT_ID_ENCRYPTED
None,25348437973,0,0,F,100,1599280715,1599705075,1599618695,86400,16,16,None,2,https://billing.wzukltd.com/ipn/paypal.php,None,Content-Type: application/x-www-form-urlencodedUser-Agent: PayPal IPN ( https://www.paypal.com/ipn ),b'mc_gross=28.00&mp_custom=&mp_currency=USD&protection_eligibility=Eligible&payer_id=86955BFBTUGJU&payment_date=21%3A38%3A17+Sep+04%2C+2020+PDT&mp_id=B-1GP27012SG5635716&payment_status=Completed&charset=windows-1252&first_name=Latrachaui&mp_status=0&mc_fee=0.78&notify_version=3.9&custom=&payer_status=verified&business=websitebuilder-paypal%40endurance.com&quantity=1&verify_sign=A.GF2oArIMOgrTQolyjukfNVpReOAX5Y3upftZr-xOVJ-Y.RXzyxQ6KE&payer_email=wickedhaircollection20%40gmail.com&txn_id=20F39644D39514533&payment_type=instant&payer_business_name=Wicked+Hair+Collection&last_name=Tyler&mp_desc=&receiver_email=websitebuilder-paypal%40endurance.com&payment_fee=0.78&mp_cycle_start=26&shipping_discount=0.00&insurance_amount=0.00&receiver_id=YSUP7PBK2P5PC&txn_type=merch_pmt&item_name=&discount=0.00&mc_currency=USD&item_number=&residence_country=US&shipping_method=Default&transaction_subject=&payment_gross=28.00&ipn_track_id=c31e3ca416da9',2,None,1897159474893678550,20002,None,20F39644D39514533


HTML(value='')

##### Search by Merchant Account Number and the Transaction ID

In [31]:
%%oracle -db QNOTIF
select * from QTOCDBA.WHTTP_MSG_QUEUE_ARCHIVE where account_number = '1897159474893678550' and PAYMENT_ID_ENCRYPTED = '20F39644D39514533'

HTML(value='')

HTML(value='')

HTML(value='')

PARTITION_ID,ID,TYPE,FLAGS,STATUS,PRIORITY,TIME_CREATED,TIME_SEND,TIME_LAST,RETRY_INTERVAL,RETRY_COUNT,RETRY_MAX,HTTP_TYPE,HTTP_FLAGS,HTTP_URL,HTTP_PROXY,HTTP_HEADERS,HTTP_BODY,HTTP_RESULT,HTTP_REPLY,ACCOUNT_NUMBER,HTTP_RESPONSE_TIME,HTTP_RESPONSE_CODE,PAYMENT_ID_ENCRYPTED
None,25348437973,0,0,F,100,1599280715,1599705075,1599618695,86400,16,16,None,2,https://billing.wzukltd.com/ipn/paypal.php,None,Content-Type: application/x-www-form-urlencodedUser-Agent: PayPal IPN ( https://www.paypal.com/ipn ),b'mc_gross=28.00&mp_custom=&mp_currency=USD&protection_eligibility=Eligible&payer_id=86955BFBTUGJU&payment_date=21%3A38%3A17+Sep+04%2C+2020+PDT&mp_id=B-1GP27012SG5635716&payment_status=Completed&charset=windows-1252&first_name=Latrachaui&mp_status=0&mc_fee=0.78&notify_version=3.9&custom=&payer_status=verified&business=websitebuilder-paypal%40endurance.com&quantity=1&verify_sign=A.GF2oArIMOgrTQolyjukfNVpReOAX5Y3upftZr-xOVJ-Y.RXzyxQ6KE&payer_email=wickedhaircollection20%40gmail.com&txn_id=20F39644D39514533&payment_type=instant&payer_business_name=Wicked+Hair+Collection&last_name=Tyler&mp_desc=&receiver_email=websitebuilder-paypal%40endurance.com&payment_fee=0.78&mp_cycle_start=26&shipping_discount=0.00&insurance_amount=0.00&receiver_id=YSUP7PBK2P5PC&txn_type=merch_pmt&item_name=&discount=0.00&mc_currency=USD&item_number=&residence_country=US&shipping_method=Default&transaction_subject=&payment_gross=28.00&ipn_track_id=c31e3ca416da9',2,None,1897159474893678550,20002,None,20F39644D39514533


HTML(value='')

##### If you want to take a look at the CAL results, look at the IPN message and find the ipn_track_id = c31e3ca416da9, which can be used to query the CAL logs.  

##### When you're querying CAL logs, look for 'queueserv-http-qtoc' as it will contain the logs for the IPN delivery. 

For More information, visit: https://engineering.paypalcorp.com/confluence/display/UNP/IPN+Documentation <br>
Also, https://engineering.paypalcorp.com/confluence/pages/viewpage.action?pageId=209752827